# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


Number of threads: 4


## Spectrum

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [10]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c_S2 = CC(β=1 / (big"0.8" + big"0.1" * im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
diag_field = Field(c_S2, r=0, s=big"0.4" + big"0.1" * im)
fields_S2 = vcat([Field(c_S2, r=r, s=s) for (r, s) in ndiag_indices], diag_field)

# Determine the parity of the number of legs in 4pt channels
function chan_parities(co::Correlation4)
    V1, V2, V3, V4 = co.fields
    chan_parities = Channels{Rational}((V1.r + V2.r) % 1, (V1.r + V4.r) % 1, (V1.r + V3.r) % 1)
end

# Compute series of blocks for any channel field compatible
# with the correlation co
function LoopSpectra(co, fields, fs)
    Vs = @channels filter(V -> V.r % 1 == chan_parities(co)[chan], fields)
    @channels ChannelSpectrum(co, chan, Vs[chan], fs[chan])
end

# Compute series of blocks, keeping only one of each (field, reflected field) pair
# given indices of the external fields.
function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 1.5 * precision)
    co = Correlation([Field(c_S2, r=r, s=s) for (r, s) in ind], Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    fs = Channels(chan -> (V -> IBlock(co, chan, V, parity=parity)))
    LoopSpectra(co, fields, fs)
end

# Solve crossing symmetry for given signature
function solve(specs, signature, even_spin=Channels(false); rmax=3, show=true)
    specs = @channels filter(V -> V.r >= signature[chan], specs[chan])
    specs = @channels filter(V -> even_spin[chan] ? spin(V) % 2 == 0 : true, specs[chan])
    system = solve_bootstrap(specs)
    if show
        show(stdout, system.str_cst, rmax=rmax)
    end
    return system
end

solve (generic function with 2 methods)

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [ ]:
ind = ((1 // 2, 0), (1 // 2, 0), (1 // 2, 0), (1 // 2, 0))
blocks_even = precompute_blocks(ind, fields_S2, parity=1, precision=10);

In [ ]:
sol = solve(blocks_even, Sig(0, 1, 1), rmax=2);

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [78]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks_even = precompute_blocks(ind, fields_S2, parity=1, precision=20);

### Signature $(0, \frac32, \frac32)$

In [79]:
sol = solve(blocks_even, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.16+0.06im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │        -0.26148+0.923384im │    5.32739e-19 │
│          (1, 1) │                          0 │        2.06519 │
│          (2, 0) │       0.0017207-0.002615im │    2.99699e-16 │
│       (2, 1//2) │                          0 │        2.18572 │
│          (2, 1) │                          0 │        2.15411 │
│          (3, 0) │ -1.28106e-06-8.33233e-08im │    3.14966e-12 │
│       (3, 1//3) │                          0 │        2.67088 │
│       (3, 2//3) │  8.01481e-07-6.16705e-08im │    1.47355e-12 │
│          (3, 1) │                          0 │        2.10467 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌──────────────┬──────────────────────┬───────────────

### Signature $(1, \frac12, \frac32)$

In [80]:
solve(blocks_even, Sig(1, 1//2, 3//2));

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │       0.502183-0.0830317im │    9.88134e-19 │
│    (2, 0) │     0.00239155+0.0255139im │    6.66122e-17 │
│ (2, 1//2) │     0.00273704+0.0200265im │    2.02014e-17 │
│    (2, 1) │                          0 │        1.28257 │
│    (3, 0) │ -6.40312e-07-1.92164e-06im │     2.5215e-12 │
│ (3, 1//3) │ -7.07647e-07-9.43683e-07im │    3.51888e-12 │
│ (3, 2//3) │ -6.00257e-07+2.35486e-07im │    2.48351e-12 │
│    (3, 1) │                          0 │         1.4421 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (1//

### Signature $(1, \frac32, \frac12)$

In [81]:
sol = solve(blocks_even, Sig(1, 3//2, 1//2));

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │      -0.502183+0.0830317im │    6.14357e-19 │
│    (2, 0) │     0.00239155+0.0255139im │     4.3717e-17 │
│ (2, 1//2) │    -0.00273704-0.0200265im │    1.22157e-17 │
│    (2, 1) │                          0 │        1.38137 │
│    (3, 0) │ -6.40312e-07-1.92164e-06im │     2.3246e-12 │
│ (3, 1//3) │  7.07647e-07+9.43683e-07im │     2.8984e-12 │
│ (3, 2//3) │ -6.00257e-07+2.35486e-07im │    2.18162e-12 │
│    (3, 1) │                          0 │        1.29062 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (3//

## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [82]:
ind = ((0, 1//2), (1, 0), (0, 1//2), (0, 1//2))
blocks_even_S2 = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks_even_S2, Sig(0, 1, 1), Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.16+0.06im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │                          0 │         1.9442 │
│          (2, 0) │  -0.000314628+0.00161483im │    7.18649e-17 │
│          (2, 1) │ -1.68652e-05-0.000641317im │    2.66416e-17 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                          0 │         1.9442 │
│    (1, 1) │  -0.000314628+0.00161483im │    7.18649e-17 │
│    (2, 0) │ -1.68652e-05-0.000641317im │    2.66416e-17 │
│ (2, 1//2) │                          0 │        1.69318 │
│    (2, 1) │                 

## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [93]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks_even_S2 = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks_even_S2, Sig(1, 1, 1), Channels(true, false, false), rmax=4);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (2, 0) │      0.0449524+0.0767135im │       0.318693 │
│    (2, 1) │     -0.0190418-0.0159205im │       0.318693 │
│    (3, 0) │ -6.59937e-05+5.41536e-05im │     1.15741e-8 │
│ (3, 2//3) │  1.83437e-05-1.91256e-05im │     6.09016e-9 │
│    (4, 0) │                          0 │       0.304527 │
│ (4, 1//2) │                          0 │       0.865294 │
│    (4, 1) │                          0 │       0.284313 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │      0.0449524+0.0767135im │       0.318693 │
│    (1, 1) │     

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

In [ ]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks_even_S2 = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks_even_S2, Sig(1, 1, 1), Channels(true, false, false), rmax=4);

# Torus one point functions

In [59]:
import EllipticFunctions: etaDedekind
import BootstrapVirasoro: PosCache, xfromq, qfromx
const Sig = Channels{Rational}

c = CC(β=sqrt(big"2")/(big"0.8"+big"0.1"*im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r*s % (1 // 2) == 0]
diag_field = Field(c, r=0, s=big"0.4"+big"0.1"*im)
fields = vcat([Field(c, r = r, s = s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(index::NTuple{2, <:Number}, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2*precision), base=10)
    Δmax = floor(Int, 1.5*precision)
    co = Correlation(Field(c, r=index[1], s=index[2]), Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

xfromτ(τ) = xfromq(exp(im * (π * τ)))

function prefactor_f(τ, co_s, chan, lr)
    x_cache = PosCache(xfromτ(τ), co_s[lr], chan)
    return inv(
        etaDedekind(τ) * x_cache.prefactor
    )
end

nonchiral_prefactor_f(τ, co_s, chan) = prefactor_f(τ, co_s, chan, :left) * 
    prefactor_f(-conj(τ), co_s, chan, :right)

relative_error(a, b) = abs((a - b) / (a + b))

relative_error (generic function with 1 method)

In [89]:
ind = (1, 0)
ind_S2 = ((0, 1 // 2), (ind[1], ind[2] // 2), (0, 1 // 2), (0, 1 // 2))
blocks = precompute_blocks(ind, fields, parity=0, precision=15)
blocks_S2 = precompute_blocks(ind_S2, fields_S2, parity=0, precision=10)

for V in keys(blocks.s.blocks)
        V_S2 = Field(c_S2, r=2 * V.r, s=V.s)
        if V_S2 in keys(blocks_S2.s.blocks)
                b = blocks.s.blocks[V]
                b_S2 = @channels blocks_S2[chan].blocks[V_S2]

                P = b.chan_field.dims.left.P
                Pbar = b.chan_field.dims.right.P
                τ = big"0.36" + big"1.14" * im
                x = xfromq(exp(im * (π * τ)))
                ts = Channels(τ, -1/τ, (τ-2)/(τ-1))
                xs = Channels(x, 1-x, 1/x)

                for chan in BootstrapVirasoro.CHANNELS
                        err = relative_error(b(ts[chan]) * 16^(2P^2 + 2Pbar^2),
                                b_S2[chan](xs[chan]) * nonchiral_prefactor_f(ts[chan], co_S2, chan) *
                                (chan === :u ? (-1)^(spin(V_S2)) : 1))
                        if err > 1e-5
                                println(V, ", $chan, ", err)
                        end
                end
        end
end

(3//2, 1), s, 0.33893471661986
(3//2, 1), t, 0.50327776110407
(3//2, 1), u, 0.36421276243345
(2, 3//4), t, 0.00011644695932445
(2, 3//4), u, 0.0012080750572938
(2, 1//2), t, 2.847678688872e-05
(2, 1//2), u, 0.00021116812645605
(2, -1//2), t, 2.8205514333302e-05
(2, -1//2), u, 0.00021333225019116
(1//2, 1), s, 1.0922758627075
(1//2, 1), t, 0.60697496133162
(1//2, 1), u, 1.1048226798684
(2, 0), u, 2.4066757610602e-05
(2, -3//4), t, 0.00011563648062607
(2, -3//4), u, 0.00122605472035
(2, 1//4), u, 5.0376751559222e-05
(2, -1//4), u, 5.0609755329778e-05


## r = 1

In [30]:
blocks_even = precompute_blocks((1, 0), fields, parity=0, precision=20);

In [31]:
sys = solve(blocks_even, Sig(1//2, 0, 1//2));

 Channel s
┌───────────────┬────────────────────────────┬────────────────┐
│ Field         │ Structure constant         │ Relative error │
├───────────────┼────────────────────────────┼────────────────┤
│     (1//2, 0) │         -3.84331-4.52402im │        2.22468 │
│     (1//2, 1) │      0.00449652-0.340755im │       0.458916 │
│        (1, 0) │          6092.54-5737.21im │        7.03432 │
│     (1, 1//2) │          -1316.2-6996.26im │        3.37096 │
│    (1, -1//2) │          2516.44+8161.61im │        21.0914 │
│        (1, 1) │          -9357.83-770.39im │        7.62188 │
│     (3//2, 0) │ -1.98686e+09+2.43799e+09im │        3.49671 │
│ (3//2, -1//3) │  3.22437e+09-1.50845e+09im │         3.1885 │
│  (3//2, 1//3) │  5.13646e+09-3.04533e+09im │        16.1527 │
│ (3//2, -2//3) │ -1.85632e+09+3.18785e+09im │        4.57277 │
│  (3//2, 2//3) │  -4.6459e+09-1.55148e+09im │        8.10183 │
│     (3//2, 1) │   4.18174e+07+7.5535e+07im │        2.36945 │
│        (2, 0) │  1.23351e+2

## r = 2

In [205]:
blocks_even = precompute_blocks((2, 0), fields, parity=1, precision=25);

In [206]:
sys = solve(blocks_even, Sig(1//2, 1//2, 1));

Channel s
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (1//2, 0) │                  1.0+0.0im │            0.0 │
│       (1, 0) │         -370.247-6589.17im │       0.346136 │
│       (1, 1) │         -551.224-4312.16im │        0.35027 │
│    (3//2, 0) │  6.14453e+06-2.00515e+06im │       0.379677 │
│ (3//2, 2//3) │  5.83822e+06-1.87505e+06im │       0.374344 │
│       (2, 0) │  3.13349e+10+1.63396e+10im │       0.436682 │
│    (2, 1//2) │  3.75186e+10+1.60223e+10im │       0.354676 │
│       (2, 1) │  2.95116e+10+1.50891e+10im │       0.428881 │
│    (5//2, 0) │  1.44027e+15+1.01898e+15im │       0.457327 │
│ (5//2, 2//5) │   1.71854e+15+1.0154e+15im │       0.347922 │
│ (5//2, 4//5) │  1.51624e+15+9.93452e+14im │        0.40509 │
│       (3, 0) │  1.01213e+22-1.62988e+22im │       0.642458 │
│    (3, 1//3) │  -8.4379e+21+1.63741e+22im │

## r = 3

In [ ]:
blocks_even = precompute_blocks((3, 0), fields, parity=1, precision=40);

In [ ]:
sys = solve(blocks_even, Sig(1//2,));

## r = 4